# Forecasting the All-NBA Team 
## by Chelsea Shu

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.ensemble import ExtraTreesClassifier

# Cleaning the Data

### All-NBA team from 1988-1989 to 2016-2017

This dataset has a list of players who have made the All-NBA team

In [2]:
all_nba = pd.read_csv('All.NBA.1984-2018.csv')
print (all_nba.shape)
all_nba.columns = all_nba.iloc[0]
all_nba = all_nba.drop(0)
all_nba.head()
all_nba.reset_index(0)
all_nba.tail()


FileNotFoundError: [Errno 2] File b'All.NBA.1984-2018.csv' does not exist: b'All.NBA.1984-2018.csv'

#### Cleaning the All-NBA data

In [ ]:
string = all_nba['Season'].str[0:2].astype(str) + all_nba['Season'].str[-2:].astype(str)

for i,num in enumerate(string):
    if num == '1900':
        string[i+1] = '2000'

all_nba['Season'] = string.astype(int)
all_nba = all_nba.drop(all_nba[all_nba["Season"] < 1989].index)
all_nba.reset_index(inplace = True)
all_nba.Season = all_nba.Season.astype(int).astype(str)
all_nba.Age = all_nba.Age.astype(int).astype(str)
all_nba.head()
all_nba.shape

Upon initial glance at the data for all seasons, which is shown below, I notice the dates are listed as a single year. To be consistent, I change the dates in the "all_nba" dataset to be single years. I assign the years to be the second year in the date range.  

In [ ]:
#check for null values
all_nba.isnull().sum()
all_nba.head()

### Player stats data each season from 1988-1989 to 2016-2017

Given that my dataset is from 1950-2017, I select player statistics from 1989-2017. The reason for this is because the 1988-1989 season is the first season where 15 players are selected to the All-NBA team, rather than 10 players. 


In [ ]:
season_stats = pd.read_csv("players.csv")


In [ ]:
season_stats = pd.read_csv("players.csv")
season_stats = season_stats.drop("Unnamed: 0", axis =1)
indexnames = season_stats[season_stats['Year'] <= 1988.0].index
season_stats = season_stats.drop(indexnames)
season_stats = season_stats.dropna(how = 'all')
season_stats.reset_index(inplace = True)
season_stats = season_stats.drop(columns = 'index')
season_stats.Year = season_stats.Year.astype(int).astype(str)
season_stats.Age = season_stats.Age.astype(int).astype(str)
season_stats.head()

Then, I check for null values in the dataset.

In [ ]:
#check for null values
season_stats.isnull().sum()

It seems there are two columns that are completely filled with null values, so I remove those.

In [ ]:
#remove blank columns
season_stats.drop(['blanl', 'blank2'], axis = 1, inplace = True)
season_stats.isnull().sum()

I notice that PER and USG% (which are both important basketball statistics) each have 5 null values, so I explore further to see if there is a relationship between both of these columns have null values.  It actually turns out that there are null values in USG% and PER for players that did not play any minutes throughout the season. Since they did not play, I can remove them from the data.

In [ ]:
#check USAGE and PER because these are important, explore why they are null

null_columns = season_stats.columns[season_stats.isnull().any()]
null_PER = season_stats[season_stats["PER"].isnull()][null_columns]
null_USG = season_stats[season_stats["PER"].isnull()][null_columns]
null_USG.equals(null_PER)



In [ ]:
#explore why PER is Nan and USG is Nan
season_stats.loc[season_stats['PER'].isnull()]

season_stats = season_stats.drop(season_stats.loc[season_stats['PER'].isnull()].index)
season_stats.isnull().sum()

There are some players that have only played a few games or few minutes and they have been able to accumulate stats from their time played in blowout games. This kind of data does not help to predict who gets selected to the All-NBA team, so I will remove any players that played less than 15% of the 82 games in the season and less than 10% of the total minutes in a season. 

In [ ]:
season_stats = season_stats[season_stats["G"] >= .15*82]
season_stats = season_stats[season_stats["MP"] > (.1 * 48 * 82)]
season_stats.head()

In [ ]:
#checking for null values again
season_stats.isnull().sum()

I want to explore more why the 3 point percentage is null, so I take a look at the "3P" and "3PA" variables where "3P%" is null. Once I print out the values, I can see that both "3P" and "3PA" are zeros, meaning that "3P%" is zero, but it errors and puts "NaN" because of the division by 0. So in order to fix this, since we know the 3-pointer percentage is 0, we can replace the NaN values with 0.

In [ ]:
null_3ppercent = season_stats[season_stats["3P%"].isnull()]

print ("3PA values where 3P% is null",null_3ppercent['3PA'])
print ("3P values where 3P% is null",null_3ppercent['3P'])

In [ ]:
null_3ppercent = season_stats["3P%"]
null_3ppercent.fillna('0', inplace = True)
#double check the null values
season_stats.isnull().sum()

Now I want to add a column into the season stats data for whether a player is on the all-NBA team in a given season. To do this I create a unique identifier based on the player's basic information(i.e. name, year, team, and age). I use four columns instead of just using the name to uniquely identify the player because the player's name could show up many times for different seasons of being on the All-NBA team. Then I give players who made the All-NBA team in a given year (this information is from my all_nba table) a 1 and players who did not make the team a 0.  I then output how many all-NBA players in each year to double check that there are 15 in each year.

In [ ]:
##creating unique identifier for each unique row
unique = list(season_stats['Year'].astype(str)+season_stats['Player'].str[0:5].astype(str) + season_stats['Tm'].astype(str) + season_stats['Age'].astype(str))
unique_all_nba = list(all_nba['Season'].astype(str)+all_nba['Player'].str[0:5].astype(str) + all_nba['Tm'].astype(str) + all_nba['Age'].astype(str))

season_stats["ID"] = unique
all_nba["ID"]= unique_all_nba

nba_list = []
for i in unique:
    if i in unique_all_nba:
        nba_list.append(1)
    else:
        nba_list.append(0)
        
season_stats["all_nba"] = nba_list

grouped = season_stats.groupby(["Year"]).all_nba.value_counts()
grouped

In [ ]:
season_stats[season_stats["Year"] == "1990"][season_stats["Player"] == "Chris Welp"]
season_stats.loc[862, 'all_nba'] =0
season_stats.loc[862]

season_stats[season_stats["Year"] == "2013"][season_stats["Player"] == "James Harden"]
season_stats.loc[12754, 'all_nba'] = 0

grouped = season_stats.groupby(["Year"]).all_nba.value_counts()
grouped


In [ ]:
##adding a column for if a player made the all-nba team the previous year
season_stats["last_year"] = 0

for player in season_stats["Player"]:
    b = season_stats[season_stats["Player"]== player][season_stats["all_nba"] == 1]["Year"]
    a = season_stats[season_stats["Player"]== player]["Year"]

    for i in a.index:
        if (str(int(a[i])-1)) in b.values:
            season_stats.loc[i, "last_year"] = 1
        else:
            season_stats.loc[i, "last_year"] = 0

season_stats.head()

In [ ]:
#hard coding that these ten players made it the year before because we do not have data in our dataset about 1988
players_1988 = ["Charles Barkley*", "Larry Bird*","Hakeem Olajuwon*", "Michael Jordan*",  "Magic Johnson*", "Karl Malone*",
               "Dominique Wilkins*","Patrick Ewing*", "John Stockton*", "Clyde Drexler*"]
for p in players_1988:
    row = season_stats[season_stats["Player"]== p][season_stats["Year"] == "1989"].index
    for j in row:
        season_stats.loc[j, 'last_year'] = 1
season_stats.head()


In [ ]:
#making labels_list with how many all_nbas they get in lifetime
new = season_stats.sort_values(by=['Player', "Age"])
labels_list = []
data_list = []
unique_players = new["Player"].drop_duplicates()
new_df = pd.DataFrame()
for i in unique_players:
    labels_list.append(len(new[new["Player"] == i][new["all_nba"] == 1]))
    new_df = new_df.append(new[new['Player'] == i])

In [ ]:
new_df.drop(new_df[new_df['Tm'] == 'TOT'].index, inplace = True) 
new_df.head()

I have done a lot of data cleaning, so here is my final dataset that I will be using. It has statistics for every player from 1989 to 2017 and includes a column for whether they made the All-NBA team  in a particular year or not. 

In [ ]:
season_stats = new_df
season_stats.head()

# Exploratory Analysis

#### Age and Player Efficiency

Because of the nature of basketball and aging, I suspect that there might be a relationship between age and how efficient a player is. I suspect that while players are young, their performance will get better and better, but once they've reached their peak age, their performance will start to decrease. I will explore that below. To measure, I will plot the relationshp between age and PER, which is a measure of the player's performance.

In [ ]:
#checking age and PER
x = []
y = []
for i in sorted(season_stats.Age.unique()):
    year = season_stats[season_stats.Age == i]
    per = np.mean(year['PER'])
    
    x.append(i)
    y.append(per)
plt.scatter(x,y)
plt.xlabel("Age")
plt.ylabel("PER")
plt.title("Relationship between a Player's Age and PER")
plt.show()

As we can see from this graph, player performance increases until a peak age of 26, and then it decreases steadily after that.

#### Payer Efficiency and Other Features

I want to check what other features are correlated with high performance. I suspect that points, assists, rebounds, and win shares will be highly correlated with PER because the more of these metrics a player has, the more likely they perform well. Let's explore. 

In [ ]:

fig,ax = plt.subplots(2, 2, figsize = (15,15))   #subplots
ax = ax.flatten()
        
ax[0].scatter(season_stats.PTS, season_stats.PER)
ax[0].set_title("Points vs PER")
ax[0].set_xlabel("Points")
ax[0].set_ylabel("PER")

ax[1].scatter(season_stats.TRB, season_stats.PER)
ax[1].set_title("Rebounds vs PER")
ax[1].set_xlabel("Rebounds")
ax[1].set_ylabel("PER")

ax[2].scatter(season_stats.AST, season_stats.PER)
ax[2].set_title("Assists vs PER")
ax[2].set_xlabel("Assists")
ax[2].set_ylabel("PER")

ax[3].scatter(season_stats.WS, season_stats.PER)
ax[3].set_title("Win Shares vs PER")
ax[3].set_xlabel("Win Shares")
ax[3].set_ylabel("PER")

plt.subplots_adjust(left=.5, bottom=.5, right=1, top=1, wspace=.75, hspace=.75)


My graphs confirm my suspicions. Each of these variables has a positive correlation with the player's efficiency rating. 

### Feature Importance

In order to have the most accurate model, I want to pick features that contribute most to whether someone makes the All-NBA team or not.  Looking at the columns, I notice that there are columns that are calculated from other columns (i.e. Free throw percentage (FT%) is equal to the column FT (free throws) divided by FTA (free throw attempts)). Having all three columns there seems repetitive so I will take certain columns out and leave just the percentage columns, since they capture the amount of attempts as well as successful attempts. I also remove redundant columns like WS (win shares) and WS/48.  I aso notice that some columns serve purposes for only certain player positions (i.e. DRB (defensive rebounds) would not be a strong statistic for offensive player positions), so I will drop these columns and keep the column that captures the total (i.e. Total Rebounds). Removing these columns will allow my model to be the best predictive model. 

Because I want to determine the All-NBA team based on players' abilities, I will remove two columns, Value Over Replacement Player (VORP) and Box Plus/Minus (BPM). The reason for removing these columns is because these statistics are not statistics that directly describe the player's direct statistics from playing games in the NBA. Rather, they are metrics that describe a player's value. This can introduce problems because this metric can be calculated in a way that does not objectively describe the player. 

Additionally, I also use certain features such as Age and whether a player made the All-NBA team the previous year. I suspect that players that made the team the year before are more likely to make the team again, so I want to explore if this is an important feature in predicting who makes the All-NBA team..

I then explore the top ten columns that contribute the most to who make the All-NBA team.

In [ ]:
model = ExtraTreesClassifier()
columns = ['Age', 'G', 'GS', 'MP', 'PER', 'TS%',
       '3PAr', 'FTr', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%',
       'USG%',  'WS',  
       'FG%',  '3P%',  '2P%', 'eFG%',
       'FT%',  'TRB', 'AST', 'STL', 'BLK', 'TOV',
       'PF', 'PTS', 'last_year']
model.fit(season_stats[columns],season_stats['all_nba'])
#plot graph of feature importances
feat_importances = pd.Series(model.feature_importances_, index=season_stats[columns].columns)
feat_importances.nlargest(10).plot(kind='barh')
plt.show()
print (feat_importances.nlargest(10))

As we can see from our bar plot, making the All-NBA team the previous year is a very important feature in determining who makes the team. Other statistics that contribute to predicting the All-NBA team are player efficiency (PER), points, win shares, rebounds, etc. 

I will use these ten columns in my model. It is important to note that while other features such as age may seem like they correlate with whether a player makes the All-NBA team (because as a player gets older, they play worse), the differences in how players perform when they are older are reflected in their player statistics (i.e. points, rebounds, etc). For that reason, age does not contribute much to predicting the All-NBA team, but rather other statistics do. 

# Random Forest Algorithm

Because of the nature of basketball, player statistics are season-dependent. For this reason, I decide to split my test and train data using backtesting. I train my model on two seasons and predict for the following season for every season from 1989 until 2017. For example, I might train my data on the 2007 and 2008 seasons, and I will test on the 2009 season. 

Within the All-NBA team, there are three tiers of teams, each with 1 guard, 2 forwards, and 2 centers, meaning that there are 3 guards, 6 forwards, and 6 centers that are on the All-NBA team. After my model predicts probabilities that each player will make the team, I choose the 3 guards, 6 forwards, and 6 centers with the highest probabilities of making the team. 
    

In [ ]:
feature = ['GS', 'MP', 'PER', 'USG%',  'WS', 'TRB', 'AST',  'TOV', 'PTS', 'last_year']

for i in sorted(season_stats.Year.unique())[0:-2]:
    season_train_data = season_stats[season_stats.Year == i]
    season_train_data = season_train_data.append(season_stats[season_stats.Year ==  str(int(i)+1)])
    train_data = season_train_data[feature]
    
    season_test_data = season_stats[season_stats.Year == str(int(i)+2)]
    season_test_data = season_test_data.replace({'PG': 'G','SG':'G','PF':'F', "SF": 'F', 'PF-C': 'F'})
    season_test_data = season_test_data.sort_values(by=['Pos'])
    test_data = season_test_data[feature]

    test_labels = season_test_data['all_nba']
    train_labels = season_stats[season_stats.Year == i]['all_nba']
    train_labels = train_labels.append(season_stats[season_stats.Year == str(int(i)+1)]['all_nba'])
  
    f1 = []
    f1_1 = []
    f1_0 = []
    RF = RandomForestClassifier(n_estimators =100, max_depth = 2, random_state = 0)
    RF.fit(train_data, train_labels)

    predicted_prob=RF.predict_proba(test_data)
    season_test_data['predicted_prob'] = predicted_prob[:,1]

    #predicted labels based on position restraints
    team = []
    for i in season_test_data.Pos.unique():
        if i == 'C':
            pos = season_test_data[season_test_data['Pos']  == i]
            largest=pos.nlargest(3, ['predicted_prob'])['Player'].tolist()

        else:
            pos = season_test_data[season_test_data['Pos']  == i]
            largest  = pos.nlargest(6, ['predicted_prob'])["Player"].tolist()
        for i in range(len(largest)):
            team.append(largest[i])
    
    test_predicted_labels = []
    for p in season_test_data.Player:

        if p in team:
            test_predicted_labels.append(1)
        else:
            test_predicted_labels.append(0)
    f1.append(f1_score(season_test_data['all_nba'], test_predicted_labels,  average = 'weighted'))
    f1_1.append(f1_score(season_test_data['all_nba'], test_predicted_labels, pos_label =1, average = 'binary'))
    f1_0.append(f1_score(season_test_data['all_nba'], test_predicted_labels, pos_label = 0, average = 'binary'))

print ("Weighted accuracy: " + str(np.mean(f1)))
print ("Accuracy of classifing who did NOT make the All-NBA team: " + str(np.mean(f1_0)))  
print ("Accuracy of classifying who did make the All-NBA team: " + str(np.mean(f1_1)))

As we can see, when the model predicts who does not make the team, it does so with 99% accuracy. However, when it predicts who will make the All-NBA team, it predicts with 80% accuracy. The reason for this is because there are only 15 players each year that get picked to the All-NBA team. Therefore, if the model misclassifies even just one or two players, the accuracy drops immensely. 

Overall, the model does a great job. It classifies whether players will make the All-NBA team or not correctly 98.4% of the time.